In [1]:
import torch
import torch.nn.functional as F
import numpy as np
from PIL import Image



In [2]:
# ---- load and prep image (grayscale, [B,C,H,W]) ----
img_path = "1657999067686 copy.jpeg"  # or the full path to your image
img = Image.open(img_path).convert("L")  # grayscale


In [3]:
x = torch.from_numpy(np.array(img)).float() / 255.0  # [H,W] in [0,1]
x = x.unsqueeze(0).unsqueeze(0)# [1,1,H,W]
x.shape


torch.Size([1, 1, 800, 800])

In [4]:

# ---- define 3x3 Sobel kernels ----
kx = torch.tensor([[-2., 0., 1.],
                   [-2., 0., 2.],
                   [-1., 0., 1.]]).reshape(1,1,3,3)

ky = torch.tensor([[-1., -2., -1.],
                   [ 0.,  0.,  0.],
                   [ 1.,  2.,  1.]]).reshape(1,1,3,3)



In [5]:
# move kernels to same device as input
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
x = x.to(device)
kx = kx.to(device)
ky = ky.to(device)



In [6]:
# ---- convolution with padding=1 to keep size ----
gx = F.conv2d(x, kx, padding=1)
gy = F.conv2d(x, ky, padding=1)

# ---- gradient magnitude ----
mag = gx+gy



In [7]:
# ---- normalize to [0,255] and save ----
m = mag.squeeze().detach().cpu().numpy()
m = (m / (m.max() + 1e-8) * 255.0).astype(np.uint8)
out = Image.fromarray(m)
out.save("edges_sobel.jpg")
print("Saved edges to edges_sobel.jpg")

Saved edges to edges_sobel.jpg
